# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [91]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.
# from transformers import DataCollatorForSeq2Seq
from transformers import T5TokenizerFast, T5ForConditionalGeneration, T5Config
import random
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

### 2) Config file 만들기 (선택)
- 모델 생성에 필요한 다양한 매개변수 정보를 저장할 수 있습니다.  
  따라서, 코드 상에서 모델의 매개변수를 설정할 수도 있지만 독립적인 매개변수 정보 파일을 생성하여 관리할 수 있습니다.

In [92]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization") #v1
# tokenizer = AutoTokenizer.from_pretrained("noahkim/KoT5_news_summarization")
# lcw99/t5-large-korean-text-summary
# tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")


In [93]:
config_data = {
    "general": {
        "data_path": "/root/dialogue/data", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": 'EbanLee/kobart-summary-v3', # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 500,
        "decoder_max_len": 100,
        "bos_token": "<s>",
        "eos_token": f"{tokenizer.eos_token}",
        "sep_token": "<sep>",
        # "mask_token": f"{tokenizer.mask_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ["#PhoneNumber#", "#Address#", "#DateOfBirth#", 
    "#PassportNumber#", "#SSN#", "#CardNumber#", 
    "#CarNumber#", "#Email#","#Person1#", "#Person2#", "#Person3#"]
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 30,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 50,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 200,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 4,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "wandb_repo",
        "project": "dialogue",
        "name": "suhan_up"
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size" : 32,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', '<s>', f"{tokenizer.eos_token}", f"{tokenizer.pad_token}", "<sep>"]
    }
}

- 참고✅    
: wandb 라이브러리를 사용하기 위해선 entity, project, name를 지정해주어야 합니다. wandb 홈페이지에 가입한 후 얻은 정보를 입력하여 작동할 수 있습니다.

In [75]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

### 3) Configuration 불러오기

In [76]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': '/root/dialogue/data',
             'model_name': 'EbanLee/kobart-summary-v3',
             'output_dir': './'},
 'inference': {'batch_size': 32,
               'ckt_path': 'model ckt path',
               'early_stopping': True,
               'generate_max_length': 100,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>', '<sep>'],
               'result_path': './prediction/'},
 'tokenizer': {'bos_token': '<s>',
               'decoder_max_len': 100,
               'encoder_max_len': 500,
               'eos_token': '</s>',
               'sep_token': '<sep>',
               'special_tokens': ['#PhoneNumber#',
                                  '#Address#',
                                  '#DateOfBirth#',
                                  '#PassportNumber#',
                                  '#SSN#',
                                  '#CardNumber#',
                      

In [7]:
# 실험에 쓰일 데이터의 경로, 사용될 모델, 모델의 최종 출력 결과를 저장할 경로에 대해 확인합니다.
loaded_config['general']

{'data_path': '../data/translate',
 'model_name': 'facebook/bart-large',
 'output_dir': './'}

In [39]:
# 이곳에 사용자가 저장한 데이터 dir 설정하기
# loaded_config['general']['data_path'] = "data_path"

In [162]:
# 데이터 전처리를 하기 위해 tokenization 과정에서 필요한 정보들을 확인합니다.
loaded_config['tokenizer']

{'bos_token': '<s>',
 'decoder_max_len': 200,
 'encoder_max_len': 900,
 'eos_token': '</s>',
 'sep_token': '<sep>',
 'special_tokens': ['#PhoneNumber#',
  '#Address#',
  '#DateOfBirth#',
  '#PassportNumber#',
  '#SSN#',
  '#CardNumber#',
  '#CarNumber#',
  '#Email#',
  '#Person1#',
  '#Person2#',
  '#Person3#']}

In [41]:
# 모델이 훈련 시 적용될 매개변수를 확인합니다.
loaded_config['training']

{'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 3,
 'early_stopping_threshold': 0.001,
 'evaluation_strategy': 'epoch',
 'fp16': True,
 'generation_max_length': 120,
 'gradient_accumulation_steps': 1,
 'learning_rate': 1e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'cosine',
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'overwrite_output_dir': True,
 'per_device_eval_batch_size': 32,
 'per_device_train_batch_size': 50,
 'predict_with_generate': True,
 'report_to': 'wandb',
 'save_strategy': 'epoch',
 'save_total_limit': 5,
 'seed': 42,
 'warmup_ratio': 0.1,
 'weight_decay': 0.01}

In [148]:
# 모델 학습 과정에 대한 정보를 제공해주는 wandb 설정 내용을 확인합니다.
loaded_config['wandb']

{'entity': 'wandb_repo', 'name': 'preprocess,v3', 'project': 'dialogue'}

In [77]:
# (선택) 이곳에 사용자가 사용할 wandb config 설정
# loaded_config['wandb']['entity'] = "사용할 wandb repo name"
loaded_config['wandb']['name'] = "bartv3"
loaded_config['wandb']['project'] = "summary"

In [8]:
# 모델이 최종 결과를 출력하기 위한 매개변수 정보를 확인합니다.
loaded_config['inference']

{'batch_size': 32,
 'ckt_path': 'model ckt path',
 'early_stopping': True,
 'generate_max_length': 100,
 'no_repeat_ngram_size': 2,
 'num_beams': 4,
 'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>', '<sep>'],
 'result_path': './prediction/'}

### 4) 데이터 불러와서 확인해보기
- 실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.
- Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [9]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'/root/dialogue/data/samsum&train.csv'))
train_df.head()

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스


In [11]:
# validation data의 구조와 내용을 확인합니다.
# val_df = pd.read_csv(os.path.join(data_path,'dev_translated.csv'))
# val_df.head()



,id,summary,dialogue
0,13862856,Hannah needs Betty's number but Amanda doesn't...,"Hannah: Hey, do you have Betty's number?\nAman..."
1,13729565,Eric and Rob are going to watch a stand-up on ...,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...
2,13680171,Lenny can't decide which trousers to buy. Bob ...,"Lenny: Babe, can you help me with something?\r..."
3,13729438,Emma will be home soon and she will let Will k...,"Will: hey babe, what do you want for dinner to..."
4,13828600,Jane is in Warsaw. Ollie and Jane has a party....,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju..."
...,...,...,...
814,13611902-1,Benjamin didn't come to see a basketball game ...,Alex: Were you able to attend Friday night's b...
815,13820989,The audition starts at 7.30 P.M. in Antena 3.,Jamilla: remember that the audition starts at ...
816,13717193,"Marta sent a file accidentally,","Marta: <file_gif>\r\nMarta: Sorry girls, I cli..."
817,13829115,There was a meet-and-greet with James Charles ...,Cora: Have you heard how much fuss British med...


## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [28]:
# # 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
# class Preprocess:
#     def __init__(self,
#             bos_token: str,
#             eos_token: str,
#         ) -> None:

#         self.bos_token = bos_token
#         self.eos_token = eos_token
      
#     @staticmethod
#     # 실험에 필요한 컬럼을 가져옵니다.
#     def make_set_as_df(file_path, is_train = True):
#         if is_train:
#             df = pd.read_csv(file_path)
#             train_df = df[['fname','dialogue','summary']]
#             return train_df
#         else:
#             df = pd.read_csv(file_path)
#             test_df = df[['fname','dialogue']]
#             return test_df
    
    
#     # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
#     def make_input(self, dataset,is_test = False):

#         if is_test:
#             encoder_input = dataset['dialogue']
#             decoder_input = [self.bos_token] * len(dataset['dialogue'])
#             return encoder_input.tolist(), list(decoder_input)
#         else:
#             encoder_input = dataset['dialogue']
#             decoder_input = dataset['summary'].apply(lambda x : self.bos_token + str(x)) # Ground truth를 디코더의 input으로 사용하여 학습합니다.
#             decoder_output = dataset['summary'].apply(lambda x : str(x) + self.eos_token)
#             return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [8]:
# import pandas as pd
# import re

# class Preprocess:
#     def __init__(self,
#             bos_token: str,
#             eos_token: str,
#         ) -> None:

#         self.bos_token = bos_token
#         self.eos_token = eos_token
      
#     @staticmethod
#     # 실험에 필요한 컬럼을 가져옵니다.
#     def make_set_as_df(file_path, is_train = True):
#         if is_train:
#             df = pd.read_csv(file_path)
#             train_df = df[['fname','dialogue','summary']]
#             return train_df
#         else:
#             df = pd.read_csv(file_path)
#             test_df = df[['fname','dialogue']]
#             return test_df
    
#     # 문장을 전처리하는 함수입니다.
#     @staticmethod
#     def preprocess_sentence(sentence: str) -> str:
#         # sentence = sentence.lower()  # 텍스트 소문자화
#         sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', '', sentence)  # 자음과 모음 제거
#         sentence = re.sub(r'\[.*?\]', '', sentence)  # 대괄호로 둘러싸인 텍스트 제거
#         sentence = re.sub(r"([.,!?])\1+", r"\1", sentence) 
#         sentence = re.sub(r"[^가-힣a-z0-9#@,-\[\]\(\)]", " ", sentence)  # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
#         sentence = re.sub(r'[" "]+', " ", sentence)  # 여러 개의 공백을 하나의 공백으로 바꿈
#         sentence = sentence.strip()  # 문장 양쪽 공백 제거
#         return sentence
    
#     # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
#     def make_input(self, dataset, is_test = False):

#         # 전처리 적용
#         dataset['dialogue'] = dataset['dialogue'].apply(self.preprocess_sentence)
#         if not is_test:
#             dataset['summary'] = dataset['summary'].apply(self.preprocess_sentence)

#         if is_test:
#             encoder_input = dataset['dialogue']
#             decoder_input = [self.bos_token] * len(dataset['dialogue'])
#             return encoder_input.tolist(), list(decoder_input)
#         else:
#             encoder_input = dataset['dialogue']
#             decoder_input = dataset['summary'].apply(lambda x : self.bos_token + str(x)) # Ground truth를 디코더의 input으로 사용하여 학습합니다.
#             decoder_output = dataset['summary'].apply(lambda x : str(x) + self.eos_token)
#             return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [75]:
# samsum 학습용
#  import pandas as pd
# import re

# class Preprocess:
#     def __init__(self,
#                  bos_token: str,
#                  eos_token: str,
#                  sep_token: str
#                  ) -> None:

#         self.bos_token = bos_token
#         self.eos_token = eos_token
#         self.sep_token = sep_token
    
#     @staticmethod
#     def make_set_as_df(file_path, is_train=True):
#         if is_train:
#             with open(file_path, 'r', encoding='utf-8') as file:
#                 data = json.load(file)
#                 df = pd.DataFrame(data)
#             # df = pd.read_csv(file_path)
#             train_df = df[['id', 'dialogue', 'summary']]
#             return train_df
#         else:
#             df = pd.read_csv(file_path)
#             test_df = df[['fname', 'dialogue']]
#             return test_df
        
#     def add_sep_tokens(self, dialogue):
#         dialogue_with_sep = dialogue.replace('\r\n', f' {self.sep_token} ')
#         return dialogue_with_sep

#     def make_input(self, dataset, is_test=False):
#         dataset['dialogue'] = dataset['dialogue'].apply(self.add_sep_tokens)

#         if is_test:
#             encoder_input = dataset['dialogue']
#             decoder_input = [self.bos_token] * len(dataset['dialogue'])
#             return encoder_input.tolist(), list(decoder_input)
#         else:
#             encoder_input = dataset['dialogue']
#             decoder_input = dataset['summary'].apply(lambda x: self.bos_token + str(x)) 
#             decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)
        

#             return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()

SyntaxError: invalid syntax (2729850141.py, line 1)

In [78]:
import pandas as pd
import re

class Preprocess:
    def __init__(self,
                 bos_token: str,
                 eos_token: str,
                 sep_token: str
                 ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token
        self.sep_token = sep_token
    
    @staticmethod
    def make_set_as_df(file_path, is_train=True):
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname', 'dialogue', 'summary']]
            return train_df
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname', 'dialogue']]
            return test_df

    def add_sep_tokens(self, dialogue_with_sep):
        if isinstance(dialogue_with_sep, str):
            dialogue_with_sep = re.sub(r'(\r\n|\n)', f'{self.sep_token}', dialogue_with_sep)
            return dialogue_with_sep
        else:
            return''

    def make_input(self, dataset, is_test=False):
        dataset['dialogue'] = dataset['dialogue'].apply(self.add_sep_tokens)

        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            encoder_input = dataset['dialogue']
            decoder_input = dataset['summary'].apply(lambda x: self.bos_token + str(x)) 
            decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)
        

            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()

In [37]:
# import random

# class Preprocess:
#     def __init__(self, bos_token: str, eos_token: str, mask_token: str, sep_token: str) -> None:
#         self.bos_token = bos_token
#         self.eos_token = eos_token
#         self.mask_token = mask_token
#         self.sep_token = sep_token

#     @staticmethod
#     def make_set_as_df(file_path, is_train=True):
#         if is_train:
#             df = pd.read_csv(file_path)
#             train_df = df[['fname', 'dialogue', 'summary']]
#             return train_df
#         else:
#             df = pd.read_csv(file_path)
#             test_df = df[['fname', 'dialogue']]
#             return test_df

#     def add_sep_tokens(self, dialogue):
#         # 화자가 바뀔 때 SEP 토큰을 추가하는 함수
#         pattern = r'(#Person\d+#)'
#         parts = re.split(pattern, dialogue)
#         result = []
#         prev_speaker = None
#         for part in parts:
#             if re.match(pattern, part):
#                 if prev_speaker and prev_speaker != part:
#                     result.append(self.sep_token)
#                 prev_speaker = part
#             result.append(part)
#         return ''.join(result)
    
#     def apply_permute_or_infill(self, text, mask_token, mask_rate=0.15):
#         # 발화자 태그 부분은 유지하고, 그 뒤의 텍스트만 처리
#         def process_content(content, method):
#             if method == "permute":
#                 words = content.split()
#                 return ' '.join(random.sample(words, len(words)))
#             elif method == "infill":
#                 tokens = content.split()
#                 num_to_mask = int(len(tokens) * mask_rate)
#                 masked_indices = random.sample(range(len(tokens)), num_to_mask)
#                 for idx in masked_indices:
#                     tokens[idx] = mask_token
#                 return ' '.join(tokens)

#         # 발화자 태그와 그 뒤의 내용으로 분리
#         pattern = r'(#Person\d+#:)'
#         parts = re.split(pattern, text)
#         processed_parts = []

#         for i, part in enumerate(parts):
#             if i % 2 == 0:  # 발화자 태그가 아닌 부분
#                 if part.strip() == "":
#                     continue
#                 # 어떤 처리 방법을 사용할지 랜덤으로 선택
#                 method = random.choice(["permute", "infill"])
#                 processed_parts.append(process_content(part, method))
#             else:  # 발화자 태그는 그대로 유지
#                 processed_parts.append(part)
                
#         return ''.join(processed_parts)

#     def make_input(self, dataset, is_test=False):
#         dataset['original_dialogue'] = dataset['dialogue']
#         dataset['original_dialogue'] = dataset['original_dialogue'].apply(self.add_sep_tokens)

#         dataset['dialogue'] = dataset['dialogue'].apply(lambda x: self.apply_permute_or_infill(x, self.mask_token))
#         dataset['dialogue'] = dataset['dialogue'].apply(self.add_sep_tokens)
        
#         if is_test:
#             encoder_input = dataset['original_dialogue']
#             decoder_input = [self.bos_token] * len(dataset['original_dialogue'])
#             return encoder_input.tolist(), list(decoder_input)
#         else:
#             # 원본 대화
#             original_encoder_input = dataset['original_dialogue']
#             original_decoder_input = dataset['summary'].apply(lambda x: self.bos_token + str(x))
#             original_decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)

#             # 순서가 섞인 대화 또는 infill이 적용된 대화
#             processed_encoder_input = dataset['dialogue']
#             processed_decoder_input = dataset['summary'].apply(lambda x: self.bos_token + str(x))
#             processed_decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)

#             # 원본 데이터와 가공된 데이터 모두 포함
#             encoder_input = original_encoder_input.tolist() + processed_encoder_input.tolist()
#             decoder_input = original_decoder_input.tolist() + processed_decoder_input.tolist()
#             decoder_output = original_decoder_output.tolist() + processed_decoder_output.tolist()

#             return encoder_input, decoder_input, decoder_output


In [79]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.len


In [80]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    # train_file_path = os.path.join(data_path,'total_train.csv')
    train_file_path = os.path.join('/root/dialogue/data/total_train.csv')
    
    # val_file_path = os.path.join(data_path,'dev.csv')
    val_file_path = os.path.join('/root/dialogue/data/dev.csv')
    

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    # print('-'*150)
    # print(f'train_data:\n {train_data["translated_dialogue"][0]}')
    # print(f'train_label:\n {train_data["translated_summary"][0]}')

    # print('-'*150)
    # print(f'val_data:\n {val_data["translated_dialogue"][0]}')
    # print(f'val_label:\n {val_data["translated_summary"][0]}')

    encoder_input_train , decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val , decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print('-'*10, 'Load data complete', '-'*10,)

    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10,)
    return train_inputs_dataset, val_inputs_dataset

## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [81]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [82]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=config['training']['evaluation_strategy'], # evaluation strategy to adopt during training
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to'] # (선택) wandb를 사용할 때 설정합니다.
            )

    # (선택) 모델의 학습 과정을 추적하는 wandb를 사용하기 위해 초기화 해줍니다.
    wandb.init(
        # entity=config['wandb']['entity'],
        project=config['wandb']['project'],
        name=config['wandb']['name'],
    )

    # # (선택) 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
    os.environ["WANDB_LOG_MODEL"]="true"
    os.environ["WANDB_WATCH"]="false"

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [31]:
# # 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
# def load_tokenizer_and_model_for_train(config,device):
#     print('-'*10, 'Load tokenizer & model', '-'*10,)
#     print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
#     model_name = config['general']['model_name']
#     bart_config = BartConfig().from_pretrained(model_name)
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'], config=bart_config)

#     special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
#     tokenizer.add_special_tokens(special_tokens_dict)

#     generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.
#     generate_model.to(device)
#     print(generate_model.config)

#     print('-'*10, 'Load tokenizer & model complete', '-'*10,)


#     # data_collator = DataCollatorForSeq2Seq(
#     #     tokenizer=tokenizer,
#     #     model = generate_model
#     # )
#     return generate_model , tokenizer

In [73]:
# def load_tokenizer_and_model_for_train(config, device):
#     print('-'*10, 'Load tokenizer & model', '-'*10,)
#     print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
#     model_name = config['general']['model_name']
#     model_config = T5Config.from_pretrained(model_name)
    
#     tokenizer = T5TokenizerFast.from_pretrained(model_name)
#     special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
#     tokenizer.add_special_tokens(special_tokens_dict)
    
#     generate_model = T5ForConditionalGeneration.from_pretrained(config['general']['model_name'], config=model_config)
#     generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.
#     generate_model.to(device)
#     print(generate_model.config)

#     print('-'*10, 'Load tokenizer & model complete', '-'*10,)
#     return generate_model , tokenizer

In [179]:
import pandas as pd
import sentencepiece as spm

df = pd.read_csv("/root/dialouge/data/translate/train_total.csv")
df1 = pd.read_csv("/root/dialouge/data/translate/test_translated.csv")
df2 = pd.read_csv('/root/dialouge/data/translate/dev_translated.csv')
corpus = df['translated_dialogue'].tolist() + df['translated_summary'].tolist() +df1['translated_dialogue'].tolist()+df2['translated_dialogue'].tolist() + df2['translated_summary'].tolist()

# Save corpus to a text file
with open('corpus.txt', 'w', encoding='utf-8') as f:
    for line in corpus:
        f.write(str(line) + '\n')

# Train SentencePiece model
spm.SentencePieceTrainer.Train('--input=corpus.txt --model_prefix=spm --vocab_size=32000 --model_type=bpe')

sp = spm.SentencePieceProcessor(model_file='spm.model')

# Test the model
test_sentence = "Amanda baked cookies and will bring Jerry some."
encoded = sp.encode(test_sentence, out_type=str)
decoded = sp.decode(encoded)

print("Encoded:", encoded)
print("Decoded:", decoded)

Encoded: ['▁Amanda', '▁baked', '▁cookies', '▁and', '▁will', '▁bring', '▁Jerry', '▁some', '.']
Decoded: Amanda baked cookies and will bring Jerry some.


In [180]:
# def load_tokenizer_and_model_for_train(config, device):
#     print('-'*10, 'Load tokenizer & model', '-'*10,)
#     print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
#     model_name = config['general']['model_name']

    
#     bart_config = BartConfig().from_pretrained(model_name)
#     tokenizer = spm.SentencePieceProcessor(model_file='spm.model')
#     # tokenizer = AutoTokenizer.from_pretrained(model_name)
    
#     generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'], config=bart_config)

#     special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
#     tokenizer.add_special_tokens(special_tokens_dict)

#     generate_model.resize_token_embeddings(len(tokenizer))  # 사전에 special token을 추가했으므로 재구성 해줍니다.
#     generate_model.to(device)
#     print(generate_model.config)

#     print('-'*10, 'Load tokenizer & model complete', '-'*10,)

#     return generate_model, tokenizer


In [83]:
def load_tokenizer_and_model_for_train(config, device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
    model_name = config['general']['model_name']

    
    bart_config = BartConfig().from_pretrained(model_name)
    # tokenizer = spm.SentencePieceProcessor(model_file='spm.model')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'], config=bart_config)

    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer))  # 사전에 special token을 추가했으므로 재구성 해줍니다.
    generate_model.to(device)
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model, tokenizer


## 3. 모델 학습하기

In [54]:
# def main(config):
#     # 사용할 device를 정의합니다.
#     device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#     print('-'*10, f'device : {device}', '-'*10,)
#     print(torch.__version__)

#     # 사용할 모델과 tokenizer를 불러옵니다.
#     generate_model, tokenizer= load_tokenizer_and_model_for_train(config, device)
#     print('-'*10, "tokenizer special tokens : ", tokenizer.special_tokens_map, '-'*10)

#     # 학습에 사용할 데이터셋을 불러옵니다.
#     preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])
#     data_path = config['general']['data_path']
#     train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config, preprocessor, data_path, tokenizer)

#     # Trainer 클래스를 불러옵니다.
#     trainer = load_trainer_for_train(config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset)
#     trainer.train()  # 모델 학습을 시작합니다.

#     # (선택) 모델 학습이 완료된 후 wandb를 종료합니다.
#     wandb.finish()


- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [84]:
def main(config):
    # 사용할 device를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model , tokenizer = load_tokenizer_and_model_for_train(config,device)
    # print('-'*10,"tokenizer special tokens : ",tokenizer.special_tokens_map,'-'*10)

    # 학습에 사용할 데이터셋을 불러옵니다.
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'], config['tokenizer']['sep_token']) # decoder_start_token: str, eos_token: str
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config,preprocessor, data_path, tokenizer)

    # Trainer 클래스를 불러옵니다.
    trainer = load_trainer_for_train(config, generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.

    # (선택) 모델 학습이 완료된 후 wandb를 종료합니다.
    wandb.finish()

In [85]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : EbanLee/kobart-summary-v3 ----------


config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

...

---------- Make training arguments complete ----------
---------- Make trainer ----------
---------- Make trainer complete ----------


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,0.866900,0.639214,0.328429,0.104352,0.312359
2,0.621200,0.589484,0.357580,0.123799,0.341672
3,0.573000,0.560398,0.366149,0.129279,0.345534
4,0.536500,0.543899,0.375323,0.135360,0.357153
5,0.506900,0.535035,0.376557,0.137205,0.357020
6,0.483100,0.529704,0.371931,0.137560,0.355705
7,0.461900,0.525603,0.375567,0.141111,0.358251
8,0.443900,0.523452,0.374802,0.139705,0.357182
9,0.426900,0.521803,0.375875,0.145028,0.361030
10,0.411500,0.521661,0.379832,0.142472,0.361088


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   #Person1# 은 #Person2# 에게 감기에 걸렸다고 말하고, #Person2# 는 #Person1# 에게 폐 전문의에게 천식에 대한 검사를 받게 할 것이라고 말합니다.                                                                                                                                                                      
GOLD:  #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                               
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:    지미는 헤이의 운동에 대해 #Person1# 에게 말하고, #Person1# 은 #Person1# 에게 금요일에 다리를 할 수 있다고 말합니다.                                                                                                                                                                          
GOLD:  #Pers

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


eval/loss,█▅▃▂▂▁▁▁▁▁▁▁▁
eval/rouge-1,▁▅▆▇▇▇▇▇▇████
eval/rouge-2,▁▄▅▆▆▆▇▆▇▇███
eval/rouge-l,▁▅▅▇▇▇▇▇▇▇███
eval/runtime,█▁▁▁▁▁▁▂▁▁▁▂▁
eval/samples_per_second,▁██████▇▇██▇█
eval/steps_per_second,▁██████▇▇██▇█
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/learning_rate,▁▅████▇▇▇▆▆▅▅
train/loss,█▅▄▃▃▃▂▂▂▂▁▁▁


## 4. 모델 추론하기

In [86]:
# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "/root/dialogue/baseline/checkpoint-7501"

- test data를 사용하여 모델의 성능을 확인합니다.

In [87]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config,preprocessor, tokenizer):

    # test_file_path = os.path.join(config['general']['data_path'],'test.csv')
    test_file_path = os.path.join('/root/dialogue/data/test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    test_id = test_data['fname']

    print('-'*150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-'*150)

    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    return test_data, test_encoder_inputs_dataset

In [88]:
# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model = BartForConditionalGeneration.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model , tokenizer

In [89]:
# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    generate_model , tokenizer= load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'],config['tokenizer']['sep_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)

    return output

In [90]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : EbanLee/kobart-summary-v3 ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 

100%|██████████| 16/16 [00:23<00:00,  1.44s/it]


In [22]:
output['summary']  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

0         더슨 씨는 실장에게 모든 직원에게 내부 메모가 전달되어야 하며, 모든 직원들이...
1        #Person2# 는 교통 체증에 걸렸다. #Person1# 는 #Person2...
2         케이트는 마샤와 히어로가 2개월 동안 별거 중이다가 이혼을 신청했다고 #Per...
3        #Person1# 은 브라이언의 생일을 축하하기 위해 파티를 열어준다. 그들은 ...
4        #Person2# 는 #Person1# 에게 올림픽 스타디움에 5000개의 좌석...
                             ...                        
494       찰리가 잭에게 새 비디오 게임에 초대하고 잭은 흥미롭다고 생각한다.      ...
495      #Person2# 는 #Person1# 에게 컨트리 음악에 관심을 가지게 된 계...
496       앨리스는 세탁기를 사용해본 적이 없습니다. #Person1# 은 그에게 기계가...
497       스티브는 매튜에게 그녀의 계약이 다음 달에 끝나기 때문에 집을 찾고 있다고 말...
498       프랭크가 벳시에게 승진하고 친구들 모두를 위한 큰 파티를 열 계획이라고 말한다...
Name: summary, Length: 499, dtype: object